<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

The nonlinear system
$$ \begin{align*}
3x_1 - \cos(x_2 x_3)-\frac{1}{2} &= 0 \\
x_1^2 - 625 x_2^2 - \frac{1}{4} &= 0 \\
\exp^{-x_1 x_2}+20x_3+\frac{10\pi-3}{3} &= 0
\end{align*}
$$
has a singular Jacobian matrix at the solution. Apply Broyden’s method with $x^{(0)} = (1, 1 − 1)^t$. Note that convergence may be slow or may not occur within a reasonable number of iterations.

The nonlinear system continuous with ex10.2.9

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -9)
MAX_ITR = 150

In [2]:
class NewtonMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [3]:
class Broyden(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1 / 2
        sol[1] = pow(x[0], 2) - 625 * pow(x[1], 2) - 1 / 4
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3) / 3
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -1250 * x[1]
        jac[1][2] = 0
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        A0 = self.jacobian(x)
        v = self.f(x)
        A = linalg.inv(A0)
        s = -A.dot(v)
        nx = x + s
        row = len(df)
        x = nx
        residual = linalg.norm(s, np.inf)
        df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]

        for k in range(2, MAX_ITR):
            w = v
            v = self.f(x)
            y = v - w
            z = -A.dot(y)
            p = -s.transpose().dot(z)
            u = s.transpose().dot(A)
            A = A + 1 / p * np.outer((s + z), u)
            s = -A.dot(v)
            nx = x + s
            residual = linalg.norm(s, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)
        
        print(self.f(x))
        return df

In [4]:
x0 = np.array([1, 1, -1])
Broyden().run(x0)

[ 0. -0.  0.]


,x1,x2,x3,residual,actual-residual
0,1.00000000,1.00000000,-1.00000000,nan,1.00000000
1,0.62498925,0.49999998,-0.50808767,0.50000002,0.49999998
2,0.50388951,0.35207260,-0.51551094,0.14792739,0.35207259
3,0.50725604,0.25852781,-0.51726241,0.09354478,0.25852781
4,0.49872915,0.13800006,-0.52025697,0.12052775,0.13800006
5,0.50039334,0.09627068,-0.52116004,0.04172938,0.09627068
6,0.49975777,0.05127633,-0.52234403,0.04499435,0.05127633
7,0.50005214,0.03529654,-0.52270968,0.01597979,0.03529654
8,0.49995781,0.01848429,-0.52314233,0.01681226,0.01848429
9,0.50000873,0.01292072,-0.52327454,0.00556357,0.01292072


In [5]:
x0 = np.array([5, 5, -0.5])
Broyden().run(x0)

[-0. -0.  0.]


,x1,x2,x3,residual,actual-residual
0,5.00000000,5.00000000,-0.50000000,nan,5.00000000
1,0.17569112,2.49624111,-0.47359878,4.82430888,2.49624111
2,0.32727936,1.85490034,-0.50680031,0.64134076,1.85490035
3,0.50206056,1.06767373,-0.49481607,0.78722661,1.06767373
4,0.48488499,0.69262086,-0.50431820,0.37505287,0.69262086
5,0.49560042,0.41417276,-0.51401259,0.27844810,0.41417277
6,0.50175583,0.25970734,-0.51708500,0.15446542,0.25970735
7,0.49944325,0.16354383,-0.51949777,0.09616352,0.16354383
8,0.49996516,0.09812170,-0.52115310,0.06542213,0.09812170
9,0.50001253,0.06176450,-0.52205565,0.03635720,0.06176450
